In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import tensorflow as tf
from tensorflow import keras

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten

Using Tensorflow backend

In [ ]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE=[224,224]

train_path = '/content/drive/MyDrive/chest_xray/train'
test_path = '/content/drive/MyDrive/chest_xray/test'

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')
num_classes = len(folders)

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)

In [ ]:
test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)

In [ ]:
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

In [ ]:
model.save('/content/drive/MyDrive/chest_xray.h5')

In [ ]:
from keras.models import load_model

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [ ]:
model = load_model('/content/drive/MyDrive/chest_xray.h5')

In [ ]:
img = load_img('/content/drive/MyDrive/chest_xray/test/PNEUMONIA/p4.jpeg', target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
classes = model.predict(img_data)
result = np.argmax(classes, axis=1)


In [ ]:
if result[0] == 0.5:
    print("Person is affected by PNEUMONIA")
else:
    print("Result is normal")